In [1]:
import requests
from IPython.display import clear_output
import urllib.request
import re
import os
import urllib
from bs4 import BeautifulSoup
import shutil

In [2]:
class Cuadro:
    def __init__(self, titulo, dimensiones, museo, fecha, pais, urlImagen):
        self.titulo = titulo
        self.dimensiones = dimensiones
        self.museo = museo
        self.fecha = fecha
        self.pais = pais
        self.urlImagen = urlImagen
        
    def __str__(self):
        cadena = "Titulo: " + str(self.titulo) + "\n"
        cadena += "Dimensiones: " + str(self.dimensiones) + "\n"
        cadena += "Museo: " + str(self.museo) + "\n"
        cadena += "Fecha: " + str(self.fecha) + "\n"
        cadena += "Pais: " + str(self.pais) + "\n"
        cadena += "----------------------------------\n"
        
        return cadena

### Funciones para descargar y guardar las imágenes

In [3]:
def guardarImagen(titulo, urlImagen):
    archivo = open("imagenes/" + titulo + ".jpg","wb")
    imagen = urllib.request.urlopen(urlImagen)
    while True:
        info = imagen.read(100000)
        if len(info) < 1: 
            break
        archivo.write(info)
    archivo.close()

In [4]:
def crearCarpetaImagenes():
    if os.path.exists("imagenes"):
        shutil.rmtree("imagenes")
    os.makedirs("imagenes")

### Funciones para parsear el html y para obtener los datos de todos los cuadros

In [5]:
def leerHTML():
    url = "https://es.wikipedia.org/wiki/Anexo:Cuadros_de_Goya"
    html = urllib.request.urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')
    
    return soup

In [6]:
def obtenerURLDeLaImagen(htmlImagen):
    regex = 'src="(.*?)"'
    
    resultado = re.search(regex, htmlImagen)
    urlImagen = "https:" + resultado.groups()[0]
    
    return urlImagen

In [7]:
def eliminarEtiquetaDeTitulo(titulo):
    # Al capturar todos los titulos en algunos se quedan algunas etiquetas
    # Este método la elimina
    tituloSinEtiquetas = re.sub("<i>", "", titulo)
    tituloSinEtiquetas = re.sub("<\/i>", "", tituloSinEtiquetas)
    tituloSinEtiquetas = re.sub("<br\/>", "", tituloSinEtiquetas)
    tituloSinEtiquetas = re.sub("\/", "", tituloSinEtiquetas)
    
    return tituloSinEtiquetas

In [8]:
def obtenerDatos(cuadro):
    titulo = ""
    fecha = ""
    dimensiones = ""
    museo = ""
    pais = ""
    urlImagen = ""
    
    regexImagen = "<td>(.*)<\/td>\s?"
    regexTitulo = "<td>(?:<i>)?(?: ?<a.*?>)?(.*?)(?:<a.*>)?(?:<\/a>.*?)?(?:<\/i>)?<\/td>\s?"
    regexFecha = "<td>(?:.*?)?([0-9]{4}–?(?:[0-9]{4})?)(?:.*)?<\/td>\s?"
    regexDimensiones = "<td>(?:<a .*?>)?(.*)<\/td>\s?"
    regexMuseo = "<td>(?:<a.*?>)?(.*?)(?:<\/a>.*?)?(?: ?\(?<a.*?>\))?(?:<br ?\/>)?\s?<\/td>\s?"
    regexPais = '(?:<td>(?:<span .*?>)?(?:<span .*?>)?(?:.*)?(?:title="(.*)"))?'
    
    try:
        resultado = re.search(regexImagen + regexTitulo + regexFecha + regexDimensiones + regexMuseo + regexPais, str(cuadro))
        titulo = eliminarEtiquetaDeTitulo(resultado.groups()[1])
        fecha = resultado.groups()[2]
        dimensiones = resultado.groups()[3]
        museo = resultado.groups()[4]
        pais = resultado.groups()[5]
        urlImagen = obtenerURLDeLaImagen(resultado.groups()[0])
    except Exception as e:
        pass
    finally:
        return titulo, dimensiones, museo, fecha, pais, urlImagen

In [9]:
def agregarCuadroAlMapa(mapa, clave, valor):
    if clave in mapa:
        mapa[clave].append(valor)
    else:
        mapa[clave] = []
        mapa[clave].append(valor)

In [10]:
def obtenerTodosLosDatos(cuadrosDeLaTabla):
    titulo = ""
    fecha = ""
    dimensiones = ""
    museo = ""
    pais = ""
    
    cuadros = []
    paises = set()
    museos = set()
    cuadrosDeCadaMuseo = {}
    cuadrosDeCadaPais = {}
    
    for cuadroDeLaTabla in cuadrosDeLaTabla:
        titulo = ""
        fecha = ""
        dimensiones = ""
        museo = ""
        pais = ""
        
        titulo, dimensiones, museo, fecha, pais, urlImagen = obtenerDatos(cuadroDeLaTabla)
        cuadro = Cuadro(titulo, dimensiones, museo, fecha, pais, urlImagen)
        
        cuadros.append(cuadro)
        paises.add(pais)
        museos.add(museo)
        
        agregarCuadroAlMapa(cuadrosDeCadaMuseo, museo, cuadro)
        agregarCuadroAlMapa(cuadrosDeCadaPais, pais, cuadro)
            
    return cuadros, paises, museos, cuadrosDeCadaMuseo, cuadrosDeCadaPais

In [11]:
def leerTodosLosCuadrosPaisesYMuseos():
    soup = leerHTML()
    
    tablaDeCuadros = soup.find('table', {'class' : 'sortable wikitable'})
    cuadrosDeLaTabla = tablaDeCuadros.findAll('tr')
    return obtenerTodosLosDatos(cuadrosDeLaTabla)

In [12]:
def eliminarPaisesMalLeidos(paises):
    paises.remove(None)
    paises.remove("Niemcy (aún no redactado)")
    paises.remove("Węgry (aún no redactado)")
    paises.remove("")

In [13]:
cuadros, paises, museos, cuadrosDeCadaMuseo, cuadrosDeCadaPais = leerTodosLosCuadrosPaisesYMuseos()
eliminarPaisesMalLeidos(paises)
museos.remove('')

### Funciones de selección del tipo de búsqueda, del país y del museo

In [14]:
def seleccionarTipoDeBusqueda():
    print ("1 - Búsqueda por país")
    print ("2 - Búsqueda por museo")

    noValido = True

    seleccion = input("\nSeleccione una opción: ")

    while noValido:
        try:
         
            if int(seleccion) == 1 or int(seleccion) == 2:
                noValido = False
            else:
                raise ValueError
                
        except ValueError:
            seleccion = input("\nERROR. No es un número, vuelva a introducir: ")
        except IndexError:
            seleccion = input("\nERROR. Valor fuera de rango, vuelva a introducir: ")

    print("\nTipo de búsqueda seleccionada correctamente")
    return int(seleccion)

In [15]:
def seleccionarPais(paises):
    print ("Lista de paises:\n")

    indice = 1
    for pais in paises:
        print ("%02d" % (indice,) + " - " + str(pais))
        indice += 1

    noValido = True

    seleccion = input("\nSeleccione un pais: ")

    while noValido:

            try:
                seleccion = int(seleccion)
                paisSeleccionado = paises[seleccion - 1]
                noValido = False

            except ValueError:
                seleccion = input("\nERROR. No es un número, vuelva a introducir: ")
            except IndexError:
                seleccion = input("\nERROR. Valor fuera de rango, vuelva a introducir: ")

    print("\nPais seleccionado correctamente\n")
    return paisSeleccionado

In [16]:
def seleccionarMuseo(museos):
    print ("Lista de museos:\n")

    indice = 1
    for museo in museos:
        print ("%02d" % (indice,) + " - " + str(museo))
        indice += 1

    seleccion = input("\nSeleccione un museo: ")
    noValido = True

    while noValido:

            try:
                seleccion = int(seleccion)
                museoSeleccionado = museos[seleccion - 1]
                noValido = False

            except ValueError:
                seleccion = input("\nERROR. No es un número, vuelva a introducir: ")
            except IndexError:
                seleccion = input("\nERROR. Valor fuera de rango, vuelva a introducir: ")

    print("\nMuseo seleccionado correctamente\n")
    return museoSeleccionado

### Selección del tipo de búsqueda, del país/museo y resultados

In [17]:
def mostrarCuadros(cuadros):
    clear_output()
    crearCarpetaImagenes()
    for cuadro in cuadros:
        guardarImagen(cuadro.titulo, cuadro.urlImagen)
        print (cuadro)

In [18]:
tipoDeBusquedaSeleccionada = seleccionarTipoDeBusqueda()

1 - Búsqueda por país
2 - Búsqueda por museo

Seleccione una opción: 1

Tipo de búsqueda seleccionada correctamente


In [19]:
paisSeleccionado = ""
museoSeleccionado = ""

if tipoDeBusquedaSeleccionada == 1:
    paisSeleccionado = seleccionarPais(list(paises))
    mostrarCuadros(cuadrosDeCadaPais[paisSeleccionado])
else:
    museoSeleccionado = seleccionarMuseo(list(museos))
    mostrarCuadros(cuadrosDeCadaMuseo[museoSeleccionado])

Titulo: María Teresa de Vallabriga
Dimensiones: 151,2 x 97,8 cm
Museo: Alte Pinakothek
Fecha: 1783
Pais: Alemania
----------------------------------

Titulo: Retrato de don José Queraltó
Dimensiones: 102 × 76 cm
Museo: Neue Pinakothek
Fecha: 1802
Pais: Alemania
----------------------------------

Titulo: Don Tomás Pérez de Estala
Dimensiones: 102 × 79 cm
Museo: Kunsthalle de Hamburgo
Fecha: 1795
Pais: Alemania
----------------------------------

Titulo: Pavo desplumado
Dimensiones: 44,8 × 62,4 cm
Museo: Neue Pinakothek
Fecha: 1808
Pais: Alemania
----------------------------------

Titulo: La Junta de Filipinas (boceto)
Dimensiones: 54 × 70 cm
Museo: Gemäldegalerie
Fecha: 1815
Pais: Alemania
----------------------------------

Titulo: Hombre comiendo puerros
Dimensiones: 6,2 × 5,6 cm
Museo: Gabinete de Grabados de Dresden
Fecha: 1824–1825
Pais: Alemania
----------------------------------

Titulo: Hombre espulgando a un pequeño perro
Dimensiones: 8,9 × 8,5 cm
Museo: Gabinete de Grabados 